In [0]:
import include_utils as u
from hdfstorehelper import HDFStoreHelper
import dill
from IPython.parallel import Client
import stopwatch
import rpy2.robjects as robjects
import pandas.rpy.common as com
from pandas import DataFrame, Series
import numpy as np
import rpy2
import rpy2.robjects as ro
import traceback

In [0]:
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()
r = ro.r

In [0]:
hdf = HDFStoreHelper("gypsy_samtools12.hd5")

In [0]:
pca_data = hdf.get('pca_std_data_snps')

In [0]:
%timeit pca_data.drop("L10", axis=1)

In [0]:
%timeit pca_data.T.drop("L10").T

In [0]:
rc = Client(profile="moth")

In [0]:
dview, lview = u.get_views(rc)
len(dview)

In [0]:
def do_pca(col):
    import os, sys, socket, stopwatch, traceback
    os.environ['R_HOME'] = '/home/cfriedline/R3/lib64/R'
    os.environ['LD_LIBRARY_PATH'] = "%s/lib:%s" % (os.environ['R_HOME'], 
                                               os.environ['LD_LIBRARY_PATH'])
    import rpy2.robjects.numpy2ri
    import rpy2.robjects as ro
    rpy2.robjects.numpy2ri.activate()
    import pandas.rpy.common as com
    from pandas import DataFrame
    r = ro.r
    t = stopwatch.Timer()
    sys.path.append("/home/cfriedline/ipynb/gypsy_moth/")
    print(socket.gethostname(), col)
    from hdfstorehelper import HDFStoreHelper
    hdf = HDFStoreHelper("/data7/cfriedline/ipython/notebooks/gypsy_moth/gypsy_samtools12.hd5")
    d = hdf.get('pca_std_data_snps')
    d2 = d.drop(col, axis=1)
    prcomp = r('prcomp')
    res = prcomp(d2.values, scale=False, center=False)
    x = com.convert_robj(res.rx2("x"))
    x.index = d2.index
    path = "price_in/%s" % col
    t.stop()
    print("Done!")
    return socket.gethostname(), path, str(t), x.to_dict()


In [0]:
dview['do_pca'] = do_pca

In [0]:
jobs = lview.map_async(do_pca, pca_data.columns)

In [0]:
len(jobs)

In [0]:
jobs.progress

In [0]:
type(jobs)

In [0]:
for j in jobs:
    print(j[0:3])
    d = DataFrame(j[-1])

In [0]:
jobs.elapsed